In [5]:
# imports

import json
import os
import pandas
import psycopg2
import random
import requests
import spotipy
import time
import uuid
from MusicLibrary import MusicLibrary
from spotipy.oauth2 import SpotifyOAuth

In [2]:
# read environment variables from local config file

config_directory = os.getcwd() + '/../local/'
exec(open(config_directory+'config.py').read())

In [ ]:
# Spotify API config

scope = "playlist-read-private"
spotify_connection = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [3]:
# postgres DB config

db_connection = psycopg2.connect(
    dbname=os.environ['LOCAL_DB_DBNAME'],
    user=os.environ['LOCAL_DB_USER'],
    password=os.environ['LOCAL_DB_PASSWORD'],
    host=os.environ['LOCAL_DB_HOST'], 
    port=os.environ['LOCAL_DB_PORT'],
)
cursor = db_connection.cursor()

In [6]:
# query track data for analysis
# alternative 1

cursor.execute("""
    WITH
    
        playlist_tracks AS (
            SELECT
                track_id,
                MIN(added_at) AS min_playlist_added_at,
                COUNT(DISTINCT playlist_id) AS playlist_count
            FROM playlist_track
            GROUP BY track_id
        ),

        track_artists AS (
            SELECT
                track_id,
                COUNT(DISTINCT artist_id) AS artist_count
            FROM track_artist
            GROUP BY track_id
        )
        
    SELECT
        t.id AS track_id,
        t.name AS track_name,
        pt.min_playlist_added_at,
        pt.playlist_count,
        ta.artist_count
    FROM track AS t
        LEFT JOIN playlist_tracks AS pt
            ON t.id = pt.track_id
        LEFT JOIN track_artists AS ta
            ON t.id = ta.track_id
    ;
""")

result = cursor.fetchall()

for row in result:
    print(row)
    break

('3pyPyRSFvIOMNS3S2CeMDB', 'Jolie Fille', '2025-04-27T20:33:07Z', 1, 4)


In [7]:
# close postgres DB

cursor.close()
db_connection.close()

In [8]:
# fetch data via class object
# alternative 2

lib = MusicLibrary()
if lib.load_data_from_db() is True:
    print(lib)

KeyError: 'results'